In [45]:

from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import time
import re

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm.notebook import tqdm

In [46]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [47]:
PATH = 'C:\Program Files (x86)\chromedriver.exe'
import os
import zipfile

from selenium import webdriver

PROXY_HOST = '141.98.235.180'  # rotating proxy or host
PROXY_PORT = 45785 # port
PROXY_USER = 'Selburobaslo' # username
PROXY_PASS = 'M9d5ZbQ' # password


manifest_json = """
{
    "version": "1.0.0",
    "manifest_version": 2,
    "name": "Chrome Proxy",
    "permissions": [
        "proxy",
        "tabs",
        "unlimitedStorage",
        "storage",
        "<all_urls>",
        "webRequest",
        "webRequestBlocking"
    ],
    "background": {
        "scripts": ["background.js"]
    },
    "minimum_chrome_version":"22.0.0"
}
"""

background_js = """
var config = {
        mode: "fixed_servers",
        rules: {
        singleProxy: {
            scheme: "http",
            host: "%s",
            port: parseInt(%s)
        },
        bypassList: ["localhost"]
        }
    };

chrome.proxy.settings.set({value: config, scope: "regular"}, function() {});

function callbackFn(details) {
    return {
        authCredentials: {
            username: "%s",
            password: "%s"
        }
    };
}

chrome.webRequest.onAuthRequired.addListener(
            callbackFn,
            {urls: ["<all_urls>"]},
            ['blocking']
);
""" % (PROXY_HOST, PROXY_PORT, PROXY_USER, PROXY_PASS)


def get_chromedriver(use_proxy=False, user_agent=None):
    
    chrome_options = webdriver.ChromeOptions()
    if use_proxy:
        pluginfile = 'proxy_auth_plugin.zip'

        with zipfile.ZipFile(pluginfile, 'w') as zp:
            zp.writestr("manifest.json", manifest_json)
            zp.writestr("background.js", background_js)
        chrome_options.add_extension(pluginfile)
    if user_agent:
        chrome_options.add_argument('--user-agent=%s' % user_agent)
    driver = webdriver.Chrome(
        PATH,
        chrome_options=chrome_options)
    return driver


driver = get_chromedriver()
#use_proxy=True


C:\Users\555\AppData\Local\Temp\ipykernel_55368\1056323955.py:78: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(
C:\Users\555\AppData\Local\Temp\ipykernel_55368\1056323955.py:78: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(


In [30]:
driver.get('https://www.gazprom.ru/press/news/')

In [31]:
news = []
articles = []
elements = driver.find_elements_by_css_selector("div.news a")
for element in elements:
    temp = str(element.get_attribute("href"))
    marker_1 = re.search('tag', temp, re.IGNORECASE)
    if marker_1 == None:
        news.append(element.get_attribute("href"))
        articles.append(element.get_attribute("text"))


C:\Users\555\AppData\Local\Temp\ipykernel_55368\2509237921.py:3: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  elements = driver.find_elements_by_css_selector("div.news a")


In [32]:
cur_window = driver.current_window_handle

In [33]:
print(news)

['https://www.gazprom.ru/press/news/2022/june/article553629/', 'https://www.gazprom.ru/press/news/2022/june/article553625/', 'https://www.gazprom.ru/press/news/2022/june/article553388/', 'https://www.gazprom.ru/press/news/2022/june/article553381/', 'https://www.gazprom.ru/press/news/2022/june/article553380/']


In [34]:
texts = []

for i in news:
    temp = ''
    driver.get(i)
    time.sleep(1)

    e = driver.find_elements_by_tag_name('p')
    e = e[1:-14]
    for j in e:
        temp += str(j.text)
        
        
    
    texts.append(temp)
    time.sleep(2)


C:\Users\555\AppData\Local\Temp\ipykernel_55368\2254391474.py:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  e = driver.find_elements_by_tag_name('p')


In [35]:
df_1 = pd.DataFrame(list(zip(texts, articles)), columns=['texts', 'articles'])
df_1

,texts,articles
0,"РЕЛИЗСегодня в Санкт-Петербурге состоялись торжественные мероприятия, посвященные празднованию 350-летия со дня рождения Петра I.Председатель Правления ПАО «Газпром» Алексей Миллер, Министр культуры Российской Федерации Ольга Любимова, Полномочный представитель Президента Российской Федерации в Северо-Западном федеральном округе Александр Гуцан и Губернатор Санкт-Петербурга Александр Беглов приняли участие в открытии на Марсовом поле выставки «30 картин из жизни Петра Великого. 2022». Проект является идейным продолжением выставки, организованной на этом месте в 1872 году Александром II в честь 200-летия Петра I.Ольга Любимова, Алексей Миллер, Александр Гуцан и Александр Беглов приняли участие в открытии выставки на Марсовом полеИнициатором масштабного исторического выставочного проекта, объединившего ведущие культурные институции Петербурга, выступил «Газпром». Научно-историческая концепция была предложена Государственным музеем-заповедником «Петергоф». 14 картин выставки 1872 года сохранились в фондах Государственного Русского музея, и сегодня их цифровые копии, нанесенные на холсты, представлены на Марсовом поле. Еще 16 работ были написаны по сюжетам утраченных картин выпускниками и преподавателями Санкт-Петербургской Академии художеств имени Ильи Репина.Открытие на Марсовом поле выставки «30 картин из жизни Петра Великого. 2022»На 30 полотнах, расположенных в специально построенных для выставки павильонах, изображены основные события правления Петра I: великие военные победы, коренные преобразования жизненного уклада страны, новаторские свершения в сфере науки и образования. Проект сочетает историческое исследование и современные технологические решения. Выставка на Марсовом поле открыта для жителей и гостей города бесплатно и работает круглосуточно до 9 августа включительно.«Художественные полотна, представленные здесь, являются своего рода невидимой нитью, которая объединяет поколения и создает историческую преемственность судеб нашего Отечества. И, конечно, эти художественные полотна — знак признания перед великим именем Петра I, великого преобразователя. Для нас очень символично, что выставка восстановлена на историческом месте — на Марсовом поле, в сердце столицы российской империи, в детище Петра», — сказал Алексей Миллер.Сегодня у «Лахта Центра» также состоялось открытие памятника — «Петр I, спасающий утопающих близ Лахты».Открытие памятника — «Петр I, спасающий утопающих близ Лахты»ПАО «Газпром» принимает активное участие в подготовке и проведении торжественных мероприятий, посвященных 350-летию со дня рождения Петра I. Они организованы по всей России.В Санкт-Петербурге совместно с российскими музеями и иными организациями «Газпром» участвует в реализации ряда масштабных проектов, среди которых новая постоянная экспозиция в Государственном Эрмитаже «Галерея Петра Великого» с уникальными экспонатами петровской эпохи, в том числе с личными вещами первого российского императора; проведение XV Международного Петровского конгресса «Не чародей, а гений… Личность Петра Великого на фоне эпохи»; проект «Имперский зал» в Кунсткамере, где разместится этнографическая экспозиция «Многонародная Россия»; реставрация картины И. К. Айвазовского «Петр I при Красной горке, зажигающий костер на берегу для подачи сигнала гибнущим судам своим» из коллекции Государственного Русского музея; культурно-исследовательская программа «От города к миру. Великое посольство — проектирование будущего», реализуемая в рамках социального проекта «Газпрома» «Друзья Петербурга».",«Газпром» поздравил Санкт-Петербург с 350-летием со дня рождения Петра I
1,"РЕЛИЗСегодня в Москве под руководством Председателя Совета директоров ПАО «Газпром» Виктора Зубкова состоялся Форум «Технологическая независимость сегодня — мировое лидерство завтра».Мероприятие было организовано Ассоциацией производителей оборудования «Новые технологии газовой отрасли» и приурочено к ее 10-летнему юбилею.В работе форума приняли участие представители Министерства промы

In [36]:
pd.DataFrame(df_1).to_csv('gazprom.csv', index=False, encoding = 'cp1251', errors='ignore')

In [37]:
news = []

driver.get("https://www.rbc.ru/tags/?tag=%D0%93%D0%B0%D0%B7%D0%BF%D1%80%D0%BE%D0%BC")

elements = driver.find_elements_by_css_selector("div.search-item__wrap a")
for element in elements:
    temp = str(element.get_attribute("href"))
    
    if marker_1 == None:
        news.append(element.get_attribute("href"))

C:\Users\555\AppData\Local\Temp\ipykernel_55368\2967299499.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  elements = driver.find_elements_by_css_selector("div.search-item__wrap a")


In [38]:
print(news)
articles = []
texts = []


for i in news:

    temp = ''
    driver.get(i)
    time.sleep(1)
    
    get_article =  driver.find_elements(By.XPATH, '//h1[@class="article__header__title-in js-slide-title"]')
   
    articles.append(str(get_article[0].text))

    e = driver.find_elements_by_tag_name('p')

    for j in e:
        temp += str(j.text)
        
        
    
    texts.append(temp)
    time.sleep(2)



['https://www.rbc.ru/society/09/06/2022/62a23c649a7947ab7bf3a47d', 'https://www.rbc.ru/politics/05/06/2022/629bf6b69a79471d5ee89a5a', 'https://www.rbc.ru/business/01/06/2022/62975ce39a79473739c4b21b', 'https://www.rbc.ru/business/01/06/2022/62974f699a79472f3d271568', 'https://www.rbc.ru/business/01/06/2022/62971b689a7947131e33cbc5', 'https://www.rbc.ru/business/01/06/2022/6296ff4c9a794706f0d6bf95', 'https://www.rbc.ru/politics/31/05/2022/629635d69a7947569ec20e38', 'https://www.rbc.ru/politics/31/05/2022/62962d109a7947530eb4bbbc', 'https://www.rbc.ru/business/31/05/2022/6295b91c9a7947114dcdba76', 'https://www.rbc.ru/rbcfreenews/6294f2379a7947deb6f25fcc', 'https://www.rbc.ru/business/30/05/2022/6294dc919a7947d510bfc429', 'https://www.rbc.ru/politics/30/05/2022/6294b3419a7947bdb41c1c5b', 'https://www.rbc.ru/business/26/05/2022/628ebbf89a79477b51ab51fe', 'https://www.rbc.ru/politics/25/05/2022/628e68bc9a79476463f1962b', 'https://www.rbc.ru/business/24/05/2022/628d27f89a7947759be8ec32', 'ht

C:\Users\555\AppData\Local\Temp\ipykernel_55368\698968072.py:16: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  e = driver.find_elements_by_tag_name('p')


In [39]:
df_2 = pd.DataFrame(list(zip(texts, articles)), columns=['texts', 'articles'])
df_2

,texts,articles
0,"В Санкт-Петербурге у «Лахта Центра» открыли памятник «Петр I, спасающий утопающих близ Лахты». Об этом сообщается в Telegram-канале городского правительства.«Это большой подарок городу от «Газпрома». Петр Первый взял на себя ответственность за всю нашу Россию, за живущих в ней людей. И сегодня мы благодарны ему за то, что он дал новый импульс развитию нашей страны, основал наш город», — сказал губернатор Александр Беглов.Скульптором монумента является Степан Мокроусов-Гульельми, архитектором — Михаил Мамошин. Первый российский император изображен выходящим из бушующих волн и поддерживающим двух обессилевших матросов. Высота памятника составляет 7 м.Он был создан по мотивам памятника Леопольда Бернштама «Петр I, спасающий рыбаков близ Лахты». В основе композиции того монумента лежала легенда о причине смерти Петра I. Академик Якоб Штелин утверждал, что 5 ноября 1724 года император обнаружил у Лахты севший на мель бот (одномачтовое судно) с людьми и бросился их спасать, в результате чего простудился и заболел. Однако памятник был демонтирован в 1919 году.В Москве на ВДНХ по случаю 350-летнего юбилея со дня рождения императора была открыта мультимедийная выставка «Петр I. Рождение империи». Ее 9 июня посетил президент России Владимир Путин.Глава СК Александр Бастрыкин написал стихотворение, приуроченное к 350-летней годовщине со дня рождения Петра I. Произведение называется «Ода на Юбилей Петра Первого».Поделитесь впечатлениями",В Петербурге открыли «большой подарок от «Газпрома»
1,"Российские санкции против компании Gazprom Germania и дочерних предприятий, находящихся под ее управлением, могут обойтись потребителям газа в Германии в дополнительные €5 млрд в год, пишет Welt am Sonntag со ссылкой на неназванных представителей отрасли.Компании являлись «дочками» российского «Газпрома», однако последний прекратил в них участие с 1 апреля. 11 мая Россия ввела санкции против Gazprom Germania и ряда компаний, находящихся под ее управлением. На следующий день вице-канцлер ФРГ, министр по делам экономики и защиты климата Роберт Хабек сообщил, что бывшие дочерние предприятия «Газпрома» больше не получают газ. При этом он заверил, что рынок сможет компенсировать выпадающие поставки.Welt пишет, что переориентация на других поставщиков газа будет создавать сильную нагрузку на федеральный бюджет, так как газ будет закупаться по более высоким ценам. По данным издания, с середины мая МВт·ч природного газа обходился Германии примерно в €85, тогда как обычная стоимость до начала российской операции составляла от €20 до €30.По оценке властей, ФРГ после прекращения поставок Gazprom Germania и ее «дочкам» необходимы дополнительные 10 млн куб. м газа в день, что будет стоить около €3,5 млрд в год, утверждает издание. Остальные затраты будут приходиться на заполнения газом хранилища Реден в Нижней Саксонии. К 1 октября оно должно быть заполнено на 80%. Как пишет Welt, заполнение может идти очень медленно, поэтому у Берлина нет времени ждать, когда упадут цены на газ, и процесс нужно начинать уже сейчас.С октября для потребителей газа будет установлена новая надбавка для компенсации выросших расходов, уточняет газета.Министерство экономики и защиты климата Германии не стало комментировать возможные дополнительные траты на закупку газа, сославшись на коммерческую тайну.В мае Хабек уже заявлял, что ответные санкции России могут привести к росту трат Германии на то, чтобы обеспечить страну газом. Он пояснял, что потребуется заключать новые договоры, возможно, по более высоким ценам. Министр заверял, что у Берлина есть план замещения российских поставок.","Welt узнала, что санкции против Gazprom Germania обойдутся ФРГ в €5 млрд"
2,"Австрийская OMV заплатила «Газпрому» по новой схеме, сообщили «РИА Новости» в компании.«OMV выполнила процесс платежа, соответствующий требованиям санкций. Платеж уже был произведен и по-прежнему номинирован в евро», — сказал представитель OMV.В компании заявили, что считают платежное обязательство исполненным

In [40]:
pd.DataFrame(df_2).to_csv('rbc.csv', index=False, encoding = 'cp1251', errors='ignore')

In [41]:
news = []

driver.get("https://www.raexpert.ru/database/companies/gazprom/mentions/")

elements = driver.find_elements_by_css_selector("div.b-rankings__item a")
for element in elements:
    news.append(element.get_attribute("href"))

news = news[0:4]
print(news)

['https://www.raexpert.ru/releases/2022/feb21a', 'https://www.raexpert.ru/releases/2021/jun18a', 'https://www.raexpert.ru/releases/2021/feb19', 'https://www.raexpert.ru/releases/2014/Oct21h']


C:\Users\555\AppData\Local\Temp\ipykernel_55368\1632531362.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  elements = driver.find_elements_by_css_selector("div.b-rankings__item a")


In [42]:

articles = []
texts = []


for i in news:

    temp = ''
    driver.get(i)
    time.sleep(1)
    
    get_article =  driver.find_elements(By.XPATH, '//h1[@class="b-title -white"]')
   
    articles.append(str(get_article[0].text))

    e = driver.find_elements_by_tag_name('p')
    e = e[5:]
    e = e[: -20]
    for j in e:
        temp += str(j.text)
        
        
    
    texts.append(temp)
    time.sleep(2)


C:\Users\555\AppData\Local\Temp\ipykernel_55368\2362095959.py:15: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  e = driver.find_elements_by_tag_name('p')


In [43]:
df_3 = pd.DataFrame(list(zip(texts, articles)), columns=['texts', 'articles'])
df_3

texts  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          ПАО «Газпром» – глобальная энергетическая компания. Основные направления деятельности – геологоразведка, добыча, транспортировка, хранение, переработка и реализация газа, газового конденсата и нефти, реализация газа в качестве моторного топлива, а также производство и сбыт тепло- и электроэнергии.«Газпром» является мировым лидером по запасам и добыче природного газа. Компания поставляет газ потребителям в России и более чем в 30 стран ближнего и дальнего зарубежья, является крупнейшим экспортером газа в Европу. «Газпром» входит в тройку ведущих российских производителей нефти и газового конденсата, занимает первое место в России по выработке электрической энергии среди компаний тепловой генерации и по производству тепловой энергии.Высокий уровень организации деятельности коллегиальных органов управления оказывает положительное влияние на уровень рейтинга. В Газпроме действует Совет директоров, состоящий из 11 человек, деятельность которого регламентирована положением о Совете директоров, размещенном в открытом доступе на сайте компании. При этом сдерживающее влияние на уровень рейтинга оказывает невысокая доля независимых директоров – 27,3% (3 независимых директора из 11). Агентство отмечает появление в компании новой должности - старший независимый директор в рамках развития взаимодействия между независимыми директорами, акционерами и Председателем Совета директоров. По состоянию на 30.09.2021 при Совете директо

In [44]:
pd.DataFrame(df_3).to_csv('raexpert.csv', index=False, encoding = 'cp1251', errors='ignore')

In [49]:
def get_unique(news):
    news_unique = []
    for i in news:
        if i not in news_unique:
            news_unique.append(i)
    return(news_unique)


In [54]:
news = []

driver.get("https://www.kommersant.ru/theme/731?page=2")

elements = driver.find_elements_by_css_selector("h2.uho__name rubric_lenta__item_name a")
for element in elements:
    news.append(element.get_attribute("href"))

#news = get_unique(news)
print(news)

[]


C:\Users\555\AppData\Local\Temp\ipykernel_55368\2803320889.py:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  elements = driver.find_elements_by_css_selector("div.uho__text rubric_lenta__item_text a")


In [ ]:

articles = []
texts = []


for i in news:

    temp = ''
    driver.get(i)
    time.sleep(1)
    
    get_article =  driver.find_elements(By.XPATH, '//h1[@class="doc_header__name js-search-mark"]')
   
    articles.append(str(get_article[0].text))

    e = driver.find_elements(By.XPATH, '//p[@class="doc__text"]')
    for j in e[0]:
        temp += str(j.text)
        
        
    
    texts.append(temp)
    time.sleep(2)


In [ ]:
df_4 = pd.DataFrame(list(zip(texts, articles)), columns=['texts', 'articles'])
df_4

In [ ]:
pd.DataFrame(df_4).to_csv('kommersant.csv', index=False, encoding = 'cp1251', errors='ignore')